# "Uploading" PDFs to Claude Via the API

One really nice feature of [Claude.ai](https://www.claude.ai) is the ability to upload PDFs. Let's mock up that feature in a notebook, and then test it out by summarizing a long PDF.

In [1]:
!curl -O https://arxiv.org/pdf/2212.08073.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2039k  100 2039k    0     0  25.2M      0 --:--:-- --:--:-- --:--:-- 25.5M


Now, we'll use the pypdf package to read the pdf. It's not identical to what Claude.ai uses behind the scenes, but it's pretty close. Note that this type of extraction only works for text content within PDFs. If your PDF contains visual elements (like charts and graphs) refer to the cookbook recipes in our [Multimodal folder](
https://github.com/anthropics/anthropic-cookbook/tree/main/multimodal) for techniques.

In [2]:
%pip install --quiet "anthropic[bedrock]" pypdf

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pypdf import PdfReader

reader = PdfReader("2212.08073.pdf")
number_of_pages = len(reader.pages)
text = ''.join([page.extract_text() for page in reader.pages])
print(text[:2155])

Constitutional AI: Harmlessness from AI Feedback
Yuntao Bai∗, Saurav Kadavath, Sandipan Kundu, Amanda Askell, Jackson Kernion,
Andy Jones, Anna Chen, Anna Goldie, Azalia Mirhoseini, Cameron McKinnon,
Carol Chen, Catherine Olsson, Christopher Olah, Danny Hernandez, Dawn Drain,
Deep Ganguli, Dustin Li, Eli Tran-Johnson, Ethan Perez, Jamie Kerr, Jared Mueller,
Jeffrey Ladish, Joshua Landau, Kamal Ndousse, Kamile Lukosuite, Liane Lovitt,
Michael Sellitto, Nelson Elhage, Nicholas Schiefer, Noemi Mercado, Nova DasSarma,
Robert Lasenby, Robin Larson, Sam Ringer, Scott Johnston, Shauna Kravec,
Sheer El Showk, Stanislav Fort, Tamera Lanham, Timothy Telleen-Lawton, Tom Conerly,
Tom Henighan, Tristan Hume, Samuel R. Bowman, Zac Hatﬁeld-Dodds, Ben Mann,
Dario Amodei, Nicholas Joseph, Sam McCandlish, Tom Brown, Jared Kaplan∗
Anthropic
Abstract
As AI systems become more capable, we would like to enlist their help to supervise
other AIs. We experiment with methods for training a harmless AI assistant

With the paper downloaded and in memory, we can ask Claude to perform various fun tasks with it.

In [4]:
from anthropic import AnthropicBedrock
client = AnthropicBedrock()
MODEL_NAME = "anthropic.claude-3-sonnet-20240229-v1:0"

In [5]:
def get_completion(client, prompt):
    return client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        messages=[{
            "role": 'user', "content":  prompt
        }]
    ).content[0].text

In [6]:
completion = get_completion(client,
    f"""Here is an academic paper: <paper>{text}</paper>

Please do the following:
1. Summarize the abstract at a kindergarten reading level. (In <kindergarten_abstract> tags.)
2. Write the Methods section as a recipe from the Moosewood Cookbook. (In <moosewood_methods> tags.)
3. Compose a short poem epistolizing the results in the style of Homer. (In <homer_results> tags.)
"""
)
print(completion)

<kindergarten_abstract>
Some smart people wanted to make robots that could help other robots. They tried a fun way to train a robot without any humans helping. The robot needed to follow some simple rules, like be nice and don't be mean. First, they asked the robot to look at things it said and tell if they were bad. Then they asked the robot to change the bad things to make them good. After that, they asked the robot to say which of two things was better. The robot learned to be very nice and helpful without ever needing a human.
</kindergarten_abstract>

<moosewood_methods>
Ingredients:
- 1 cup unbleached flour
- 1 tsp baking powder
- 1⁄2 tsp salt
- 1⁄4 cup vegetable oil or melted butter
- 1 cup milk or non-dairy milk
- 1 tbsp honey or maple syrup
- 1 egg, beaten (or egg replacer)
- 1 pretrained language model, ripe

Instructions:
1. Preheat your oven to 350°F. Grease a baking pan and set aside.
2. In a large bowl, whisk together the flour, baking powder, and salt. Make a well in the